##### Copyright 2018 The TensorFlow Authors.

In [0]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

In [0]:
#@title MIT License
#
# Copyright (c) 2017 François Chollet
#
# Permission is hereby granted, free of charge, to any person obtaining a
# copy of this software and associated documentation files (the "Software"),
# to deal in the Software without restriction, including without limitation
# the rights to use, copy, modify, merge, publish, distribute, sublicense,
# and/or sell copies of the Software, and to permit persons to whom the
# Software is furnished to do so, subject to the following conditions:
#
# The above copyright notice and this permission notice shall be included in
# all copies or substantial portions of the Software.
#
# THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
# IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
# FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL
# THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
# LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING
# FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER
# DEALINGS IN THE SOFTWARE.

# 電影評論文本分類

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://tensorflow.org/tutorials/keras/text_classification"><img src="https://tensorflow.org/images/tf_logo_32px.png" />在 tensorflow.org 上查看</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/tutorials/keras/text_classification.ipynb"><img src="https://tensorflow.org/images/colab_logo_32px.png" />在 Google Colab 中運行</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/docs/blob/master/site/en/tutorials/keras/text_classification.ipynb"><img src="https://tensorflow.org/images/GitHub-Mark-32px.png" />在 GitHub 上查看源代碼</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/docs/site/en/tutorials/keras/text_classification.ipynb"><img src="https://tensorflow.org/images/download_logo_32px.png" />下載 notebook</a>
  </td>
</table>

Note: 我們的 TensorFlow 社區翻譯了這些文檔。因為社區翻譯是盡力而為， 所以無法保證它們是最準確的，並且反映了最新的
[官方英文文檔](https://www.tensorflow.org/?hl=en)。如果您有改進此翻譯的建議， 請提交 pull request 到
[tensorflow/docs](https://github.com/tensorflow/docs) GitHub倉庫。要志願地撰寫或者審核譯文，請加入
[docs@tensorflow.org Google Group](https://groups.google.com/a/tensorflow.org/forum/#!forum/docs)。


此筆記本（notebook）使用評論文本將影評分為*積極（positive）*或*消極（nagetive）*兩類。這是一個*二元（binary）*或者二分類問題，一種重要且應用廣泛的機器學習問題。

我們將使用來源於[網絡電影數據庫（Internet Movie Database）](https://www.imdb.com/)的 [IMDB 數據集（IMDB dataset）](https://tensorflow.org/api_docs/python/tf/keras/datasets/imdb)，其包含 50,000 條影評文本。從該數據集切割出的25,000條評論用作訓練，另外 25,000 條用作測試。訓練集與測試集是*平衡的（balanced）*，意味著它們包含相等數量的積極和消極評論。

此筆記本（notebook）使用了 [tf.keras](https://tensorflow.org/guide/keras)，它是一個 Tensorflow 中用於構建和訓練模型的高級API。有關使用 `tf.keras` 進行文本分類的更高級教程，請參閱 [MLCC文本分類指南（MLCC Text Classification Guide）](https://developers.google.com/machine-learning/guides/text-classification/)。

In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals

try:
  # Colab only
  %tensorflow_version 2.x
except Exception:
    pass
import tensorflow as tf
from tensorflow import keras

import numpy as np

print(tf.__version__)

## 下載 IMDB 數據集

IMDB 數據集已經打包在 TensorFlow 中。該數據集已經經過預處理，評論（單詞序列）已經被轉換為整數序列，其中每個整數表示字典中的特定單詞。

以下代碼將下載 IMDB 數據集到您的機器上（如果您已經下載過將從緩存中復制）：

In [0]:
imdb = keras.datasets.imdb

(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words=10000)

參數 `num_words=10000` 保留了訓練數據中最常出現的 10,000 個單詞。為了保持數據規模的可管理性，低頻詞將被丟棄。


## 探索數據

讓我們花一點時間來了解數據格式。該數據集是經過預處理的：每個樣本都是一個表示影評中詞彙的整數數組。每個標籤都是一個值為 0 或 1 的整數值，其中 0 代表消極評論，1 代表積極評論。

In [0]:
print("Training entries: {}, labels: {}".format(len(train_data), len(train_labels)))

評論文本被轉換為整數值，其中每個整數代表詞典中的一個單詞。首條評論是這樣的：

In [0]:
print(train_data[0])

電影評論可能具有不同的長度。以下代碼顯示了第一條和第二條評論的中單詞數量。由於神經網絡的輸入必須是統一的長度，我們稍後需要解決這個問題。

In [0]:
len(train_data[0]), len(train_data[1])

### 將整數轉換回單詞

了解如何將整數轉換回文本對您可能是有幫助的。這裡我們將創建一個輔助函數來查詢一個包含了整數到字符串映射的字典對象：

In [0]:
# 一個映射單詞到整數索引的詞典
word_index = imdb.get_word_index()

# 保留第一個索引
word_index = {k:(v+3) for k,v in word_index.items()}
word_index["<PAD>"] = 0
word_index["<START>"] = 1
word_index["<UNK>"] = 2  # unknown
word_index["<UNUSED>"] = 3

reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

def decode_review(text):
    return ' '.join([reverse_word_index.get(i, '?') for i in text])

現在我們可以使用 `decode_review` 函數來顯示首條評論的文本：

In [0]:
decode_review(train_data[0])

## 準備數據

影評——即整數數組必須在輸入神經網絡之前轉換為張量。這種轉換可以通過以下兩種方式來完成：

* 將數組轉換為表示單詞出現與否的由 0 和 1 組成的向量，類似於 one-hot 編碼。例如，序列[3, 5]將轉換為一個 10,000 維的向量，該向量除了索引為 3 和 5 的位置是 1 以外，其他都為 0。然後，將其作為網絡的首層——一個可以處理浮點型向量數據的稠密層。不過，這種方法需要大量的內存，需要一個大小為 `num_words * num_reviews` 的矩陣。

* 或者，我們可以填充數組來保證輸入數據具有相同的長度，然後創建一個大小為 `max_length * num_reviews` 的整型張量。我們可以使用能夠處理此形狀數據的嵌入層作為網絡中的第一層。

在本教程中，我們將使用第二種方法。

由於電影評論長度必須相同，我們將使用 [pad_sequences](https://tensorflow.org/api_docs/python/tf/keras/preprocessing/sequence/pad_sequences) 函數來使長度標準化：

In [0]:
train_data = keras.preprocessing.sequence.pad_sequences(train_data,
                                                        value=word_index["<PAD>"],
                                                        padding='post',
                                                        maxlen=256)

test_data = keras.preprocessing.sequence.pad_sequences(test_data,
                                                       value=word_index["<PAD>"],
                                                       padding='post',
                                                       maxlen=256)

現在讓我們看下樣本的長度：

In [0]:
len(train_data[0]), len(train_data[1])

並檢查一下首條評論（當前已經填充）：

In [0]:
print(train_data[0])

## 構建模型

神經網絡由堆疊的層來構建，這需要從兩個主要方面來進行體系結構決策：

* 模型裡有多少層？
* 每個層裡有多少*隱層單元（hidden units）*？

在此樣本中，輸入數據包含一個單詞索引的數組。要預測的標籤為 0 或 1。讓我們來為該問題構建一個模型：

In [0]:
# 輸入形狀是用於電影評論的詞彙數目（10,000 詞）
vocab_size = 10000

model = keras.Sequential()
model.add(keras.layers.Embedding(vocab_size, 16))
model.add(keras.layers.GlobalAveragePooling1D())
model.add(keras.layers.Dense(16, activation='relu'))
model.add(keras.layers.Dense(1, activation='sigmoid'))

model.summary()

層按順序堆疊以構建分類器：

1. 第一層是`嵌入（Embedding）`層。該層採用整數編碼的詞彙表，並查找每個詞索引的嵌入向量（embedding vector）。這些向量是通過模型訓練學習到的。向量向輸出數組增加了一個維度。得到的維度為：`(batch, sequence, embedding)`。
2. 接下來，`GlobalAveragePooling1D` 將通過對序列維度求平均值來為每個樣本返回一個定長輸出向量。這允許模型以盡可能最簡單的方式處理變長輸入。
3. 該定長輸出向量通過一個有 16 個隱層單元的全連接（`Dense`）層傳輸。
4. 最後一層與單個輸出結點密集連接。使用 `Sigmoid` 激活函數，其函數值為介於 0 與 1 之間的浮點數，表示概率或置信度。

### 隱層單元

上述模型在輸入輸出之間有兩個中間層或“隱藏層”。輸出（單元，結點或神經元）的數量即為層表示空間的維度。換句話說，是學習內部表示時網絡所允許的自由度。

如果模型具有更多的隱層單元（更高維度的表示空間）和/或更多層，則可以學習到更複雜的表示。但是，這會使網絡的計算成本更高，並且可能導致學習到不需要的模式——一些能夠在訓練數據上而不是測試數據上改善性能的模式。這被稱為*過擬合（overfitting）*，我們稍後會對此進行探究。

### 損失函數與優化器

一個模型需要損失函數和優化器來進行訓練。由於這是一個二分類問題且模型輸出概率值（一個使用 sigmoid 激活函數的單一單元層），我們將使用 `binary_crossentropy` 損失函數。

這不是損失函數的唯一選擇，例如，您可以選擇 `mean_squared_error` 。但是，一般來說 `binary_crossentropy` 更適合處理概率——它能夠度量概率分佈之間的“距離”，或者在我們的示例中，指的是度量 ground-truth 分佈與預測值之間的“距離”。

稍後，當我們研究回歸問題（例如，預測房價）時，我們將介紹如何使用另一種叫做均方誤差的損失函數。

現在，配置模型來使用優化器和損失函數：

In [0]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

## 創建一個驗證集

在訓練時，我們想要檢查模型在未見過的數據上的準確率（accuracy）。通過從原始訓練數據中分離 10,000 個樣本來創建一個*驗證集*。 （為什麼現在不使用測試集？我們的目標是只使用訓練數據來開發和調整模型，然後只使用一次測試數據來評估準確率（accuracy））。

In [0]:
x_val = train_data[:10000]
partial_x_train = train_data[10000:]

y_val = train_labels[:10000]
partial_y_train = train_labels[10000:]

## 訓練模型

以 512 個樣本的 mini-batch 大小迭代 40 個 epoch 來訓練模型。這是指對 `x_train` 和 `y_train` 張量中所有樣本的的 40 次迭代。在訓練過程中，監測來自驗證集的 10,000 個樣本上的損失值（loss）和準確率（accuracy）：

In [0]:
history = model.fit(partial_x_train,
                    partial_y_train,
                    epochs=40,
                    batch_size=512,
                    validation_data=(x_val, y_val),
                    verbose=1)

## 評估模型

我們來看一下模型的性能如何。將返回兩個值。損失值（loss）（一個表示誤差的數字，值越低越好）與準確率（accuracy）。

In [0]:
results = model.evaluate(test_data,  test_labels, verbose=2)

print(results)

這種十分樸素的方法得到了約 87% 的準確率（accuracy）。若採用更好的方法，模型的準確率應當接近 95%。

## 創建一個準確率（accuracy）和損失值（loss）隨時間變化的圖表

`model.fit()` 返回一個 `History` 對象，該對象包含一個字典，其中包含訓練階段所發生的一切事件：

In [0]:
history_dict = history.history
history_dict.keys()

有四個條目：在訓練和驗證期間，每個條目對應一個監控指標。我們可以使用這些條目來繪製訓練與驗證過程的損失值（loss）和準確率（accuracy），以便進行比較。

In [0]:
import matplotlib.pyplot as plt

acc = history_dict['accuracy']
val_acc = history_dict['val_accuracy']
loss = history_dict['loss']
val_loss = history_dict['val_loss']

epochs = range(1, len(acc) + 1)

# “bo”代表 "藍點"
plt.plot(epochs, loss, 'bo', label='Training loss')
# b代表“藍色實線”
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.show()

In [0]:
plt.clf()   # 清除數字

plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.show()


在該圖中，點代表訓練損失值（loss）與準確率（accuracy），實線代表驗證損失值（loss）與準確率（accuracy）。

注意訓練損失值隨每一個 epoch *下降*而訓練準確率（accuracy）隨每一個 epoch *上升*。這在使用梯度下降優化時是可預期的——理應在每次迭代中最小化期望值。

驗證過程的損失值（loss）與準確率（accuracy）的情況卻並非如此——它們似乎在 20 個 epoch 後達到峰值。這是過擬合的一個實例：模型在訓練數據上的表現比在以前從未見過的數據上的表現要更好。在此之後，模型過度優化並學習*特定*於訓練數據的表示，而不能夠*泛化*到測試數據。

對於這種特殊情況，我們可以通過在 20 個左右的 epoch 後停止訓練來避免過擬合。稍後，您將看到如何通過回調自動執行此操作。